## (3)조건에 따라 시리즈 값을 변경
### 데이터 : 지표온도와 미국 종단조사

In [5]:
import pandas as pd
import numpy as np
nls97 = pd.read_csv('data/nls97b.csv')
nls97.set_index('personid', inplace=True)
landtemps = pd.read_csv('data/landtemps2019avgs.csv')

In [7]:
# Elevation 값의 분포 빠르게 확인
landtemps.elevation.quantile(np.arange(0.2, 1.1, 0.2))

0.2      48.0
0.4     190.5
0.6     393.2
0.8    1066.8
1.0    9999.0
Name: elevation, dtype: float64

### 넘파이 where 함수로 두 값을 포함하는 범주 시리즈를 생성
- where(검사조건, 참일 경우 수행할 문장, 거짓일 경우 수행할 문장)

In [8]:
landtemps['elevation_group'] = np.where(landtemps.elevation > landtemps.elevation.quantile(0.8), 'High', 'Low')

In [10]:
landtemps.elevation_group = landtemps.elevation_group.astype('category')

In [11]:
landtemps.groupby(['elevation_group'])['elevation'].agg(['count', 'min', 'max'])

,count,min,max
elevation_group,,,
High,2409,1067.0,9999.0
Low,9686,-350.0,1066.8


### 📍where 메서드로 세 값을 포함하는 범주형 시리즈 생성
- 80번쨰 백분위수보다 높은 값을 high, 중앙값을 초과하고 80번째 백분위수 이하인 값 Median, 그 외의 값을 low로
> - 검사 안에 검사를 중첩하는 경우로, False 대신에 또 다른 where문을 넣는다 <br>
> - where(검사조건, 참일 경우 수행할 문장, where(검사조건, 참일경우 수행할 문장, 거짓일 경우 수행할 문장))

In [27]:
landtemps.elevation.median()

271.3

In [31]:
landtemps['elevation_group'] = np.where(landtemps.elevation > landtemps.elevation.quantile(0.8),'High',
                                        np.where(landtemps.elevation > landtemps.elevation.median(), 'Medium', 'Low'))

In [32]:
landtemps.elevation_group = landtemps.elevation_group.astype('category')

In [34]:
landtemps.groupby(['elevation_group'])['elevation'].agg(['count', 'min', 'max'])

,count,min,max
elevation_group,,,
High,2409,1067.0,9999.0
Low,6056,-350.0,271.3
Medium,3630,271.4,1066.8


### 📍넘파이 select 메서드를 사용해 조건의 리스트를 평가
> where문을 복잡하게 만드는 것보다, 이 방법을 추천
- 검사 조건 리스트와 결과를 담을 리스트를 각각 준비
- 여러 검사를 만족하더라도 첫 번째 참인 것만 사용됨!
    - GPA가 2미만인 개인과 학위가 없는 개인을 첫번째 범주에 넣기
    - 학위는 없지만 GPA가 높은 개인을 두 번째 범주에 넣기
    - 학위는 있지만 GPA가 낮은 개인을 세 번째 범주에 넣기
    - 나머지를 네번째 범주에 넣기

In [13]:
test = [(nls97.gpaoverall < 2) & (nls97.highestdegree == '0. None'),
        nls97.highestdegree == '0. None', nls97.gpaoverall < 2]

result = ['1. Low GPA and No Diploma', '2. No Diploma', '3. Low GPA']

In [14]:
nls97['hsachieve'] = np.select(test, result, '4.Did Okay')

In [15]:
nls97[['hsachieve', 'gpaoverall', 'highestdegree']].head()

,hsachieve,gpaoverall,highestdegree
personid,,,
100061,4.Did Okay,3.06,2. High School
100139,4.Did Okay,NaN,2. High School
100284,2. No Diploma,NaN,0. None
100292,4.Did Okay,3.45,4. Bachelors
100583,4.Did Okay,2.91,2. High School


In [17]:
nls97.hsachieve.value_counts().sort_index()

1. Low GPA and No Diploma      95
2. No Diploma                 858
3. Low GPA                    459
4.Did Okay                   7572
Name: hsachieve, dtype: int64

### 📍lambda를 사용해 여러 개의 열을 한 문장으로 테스트
> 로직이 더 복잡해지면 apply를 사용한다. <br>
> axis=1을 지정하면 데이터프레임의 각 행을 함수에 보냄
- colenr열에는 2월과 10월의 개인별 학적 상태가 있다 ->> 이 열에 3. 4-year college값이 있는지 검사하려고 한다.
    - filter를 사용해 colenr열의 데이터프레임을 생성
    - apply를 사용해 lambda 함수를 호출
    - colenr열의 첫번째 문자를 검사한다(3인지만 확인)->> 결과는 any에 전달되어 3이 첫 글자인 열이 존재하는지 평가

In [18]:
nls97.loc[[100292, 100583, 100139], 'colenrfeb00':'colenroct04'].T

personid,100292,100583,100139
colenrfeb00,1. Not enrolled,1. Not enrolled,1. Not enrolled
colenroct00,3. 4-year college,1. Not enrolled,1. Not enrolled
colenrfeb01,3. 4-year college,1. Not enrolled,1. Not enrolled
colenroct01,3. 4-year college,3. 4-year college,1. Not enrolled
colenrfeb02,3. 4-year college,3. 4-year college,1. Not enrolled
colenroct02,3. 4-year college,1. Not enrolled,1. Not enrolled
colenrfeb03,3. 4-year college,1. Not enrolled,1. Not enrolled
colenroct03,3. 4-year college,1. Not enrolled,1. Not enrolled
colenrfeb04,3. 4-year college,1. Not enrolled,1. Not enrolled
colenroct04,1. Not enrolled,1. Not enrolled,1. Not enrolled


In [19]:
nls97['baenrollment'] = nls97.filter(like='colenr').apply(lambda x: x.str[0:1]=='3').any(axis=1)

In [21]:
nls97.loc[[100292, 100583, 100139], ['baenrollment']].T

personid,100292,100583,100139
baenrollment,True,True,False


In [22]:
nls97.baenrollment.value_counts()

False    5085
True     3899
Name: baenrollment, dtype: int64

### 📍여러 시리즈의 값을 바탕으로 값을 할당하는 함수 생성
- 야간에 수면을 6시간 미만으로 취하는 이유에 따라 조사 응답자를 범주화하는 함수 생성
    - 취업 상태, 함께 거주하는 자녀 수, 임금소득, 최고 성적에 대한 응답을 바탕으로 구분

In [23]:
def getsleepdeprivedreason(row):
    sleepdeprivedreason = 'Unknown'
    if (row.nightlyhrssleep >= 6):
        sleepdeprivedreason = 'Not Sleep Deprived'
    elif (row.nightlyhrssleep > 0):
        if (row.weeksworked16 + row.weeksworked17 < 80): #거의 일을 하지 않고
            if (row.childathome > 2): # 3명 이상의 자녀와 함께 거주한 경우
                sleepdeprivedreason = 'Child Rearing' # 자녀 양육
            else:
                sleepdeprivedreason = 'Other Reasons' # 기타
        else: # 대부분 일을 한 경우
            if (row.wageincome >= 62000 or row.highestgradecompleted >= 16): #급여가 높거나 4년제 졸업
                sleepdeprivedreason = 'Work Pressure' # 직장 스트레스
            else:
                sleepdeprivedreason = 'Income Pressure' # 금전적 스트레스
    else:
        sleepdeprivedreason = 'Unknown'
    return sleepdeprivedreason

In [24]:
nls97['sleepdeprivedreason'] = nls97.apply(getsleepdeprivedreason, axis = 1)

In [25]:
nls97.sleepdeprivedreason = nls97.sleepdeprivedreason.astype('category')

In [26]:
nls97.sleepdeprivedreason.value_counts()

Not Sleep Deprived    5595
Unknown               2286
Income Pressure        462
Work Pressure          281
Other Reasons          272
Child Rearing           88
Name: sleepdeprivedreason, dtype: int64